In [108]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% ! important;}
div.cell.code_cell.rendered{width:100%}
div.input_prompt{padding:0px}
div.CodeMirror {font-family:Consolas ; font-size:12pt;}
div.text_cell_render.rendered_html {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold}
div.input {font-family:Consolas ; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper {padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="red">웹데이터 수집 주가지수</font></b>
## 주가 웹크롤링

In [212]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import date

In [220]:
# krx 전체지수 시세 창 open
driver = webdriver.Chrome()
driver.get("https://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010101")
time.sleep(2) # 데이터 로딩시간의 문제로 추가

In [159]:
# 지수명, 종가, 거래량을 각각의 리스트에 저장
krx_name_list =[]
closing_price_list =[]
trade_value_list = []
krx_list =[]

krx_name = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-LEFT[data-name="IDX_NM"]')
closing_price = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-RIGHT[data-name="CLSPRC_IDX"]')
trade_value_list = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-RIGHT[data-name="ACC_TRDVOL"]')

krx_name_list = [items.text for items in krx_name if items.text != ""]
closing_price_list = [items.text for items in closing_price if items.text != ""]
trade_value_list = [items.text for items in trade_value_list if items.text != ""]

In [160]:
# 각각의 리스트의 값들을 2차원 배열로 순서에 맞게 저장
for idx, item, in enumerate(krx_name_list):
    krx_list.append([krx_name_list[idx], closing_price_list[idx], trade_value_list[idx]])
    
krx_list

[['KTOP 30', '11,089.89', '66,383'],
 ['KRX 100', '8,871.38', '114,359'],
 ['KRX 자동차', '2,515.84', '12,711'],
 ['KRX 반도체', '5,762.35', '38,891'],
 ['KRX 헬스케어', '4,861.55', '29,995'],
 ['KRX 은행', '1,297.65', '7,938'],
 ['KRX 에너지화학', '2,582.16', '6,508'],
 ['KRX 철강', '2,491.37', '4,019'],
 ['KRX 방송통신', '795.18', '1,790'],
 ['KRX 건설', '820.60', '35,260'],
 ['KRX 증권', '1,545.79', '25,728'],
 ['KRX 기계장비', '1,480.85', '28,672'],
 ['KRX 보험', '2,627.34', '3,769'],
 ['KRX 운송', '1,197.07', '10,751'],
 ['KRX 경기소비재', '1,182.53', '9,832'],
 ['KRX 필수소비재', '1,493.49', '5,788'],
 ['KRX K콘텐츠', '1,984.57', '6,800'],
 ['KRX 정보기술', '2,403.92', '47,578'],
 ['KRX 유틸리티', '819.77', '3,709'],
 ['KRX 300 정보기술', '3,966.60', '46,365']]

In [161]:
# 필요한 키워드만 헤당 날짜와 함께 저장
keywords = ['건설', '헬스케어', '자동차'] 
fdate = driver.find_element(By.CSS_SELECTOR, 'input[name="trdDd"]')
date = fdate.get_attribute("value")

                                                                                                                 # 주말 혹은 공휴일의 데이터는 '-', 저장을 막기위해
filtered = [ [date, name, price, trade] for name, price, trade in krx_list if any(kw in name for kw in keywords) if (price != '-') or (trade != '-')] 
print(filtered)

[['20251219', 'KRX 자동차', '2,515.84', '12,711'], ['20251219', 'KRX 헬스케어', '4,861.55', '29,995'], ['20251219', 'KRX 건설', '820.60', '35,260']]


## 날짜변경

In [154]:
# 날짜 변경
new_date = "20251213"
date_input = driver.find_element(By.ID, "trdDd")
driver.execute_script("arguments[0].value = arguments[1];", date_input, new_date)

# 변경된 날짜로 조회
search_btn = driver.find_element(By.ID, "jsSearchButton") 
search_btn.click()

## 코드 합치기(미완성)

In [249]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import date
import pandas as pd

# krx 전체지수 시세 창 open
driver = webdriver.Chrome()
driver.get("https://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010101")
time.sleep(2) # 데이터 로딩시간의 문제로 추가

### 추가해야 할 부분 : 이전 달 출력변경문제

In [263]:
input_date = int(input('몇일 전 까지 크롤링할까요? '))
now_day = int(date.today().strftime("%Y%m%d"))

krx_name_list =[]
closing_price_list =[]
trade_value_list = []
krx_list =[]
result_list =[]

for date_idx in range(input_date):
    
    krx_name = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-LEFT[data-name="IDX_NM"]')
    closing_price = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-RIGHT[data-name="CLSPRC_IDX"]')
    trade_value_list = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-RIGHT[data-name="ACC_TRDVOL"]')
    
    krx_name_list = [items.text for items in krx_name if items.text != ""]
    closing_price_list = [items.text.replace(",","") for items in closing_price if items.text != ""]
    trade_value_list = [items.text.replace(",","") for items in trade_value_list if items.text != ""]
    date_maker = now_day-date_idx
    
    for idx, item, in enumerate(krx_name_list):
        krx_list.append([date_maker, krx_name_list[idx], closing_price_list[idx], trade_value_list[idx]])
    keywords = ['건설', '헬스케어', '자동차'] 
    fdate = driver.find_element(By.CSS_SELECTOR, 'input[name="trdDd"]')
    
                                                                                                                    
    result_list = [ [date, name, price, trade] for date, name, price, trade in krx_list if any(kw in name for kw in keywords) if (price != '-') or (trade != '-')] 

    # 날짜 변경
    chg_date = now_day-date_idx-1
    date_input = driver.find_element(By.ID, "trdDd")
    driver.execute_script("arguments[0].value = arguments[1];", date_input, chg_date)

    # 변경된 날짜로 조회
    search_btn = driver.find_element(By.ID, "jsSearchButton") 
    search_btn.click()
    
    time.sleep(3)
# 오늘날짜로 변경
chg_date = now_day
date_input = driver.find_element(By.ID, "trdDd")
driver.execute_script("arguments[0].value = arguments[1];", date_input, chg_date)

# 변경된 날짜로 조회
search_btn = driver.find_element(By.ID, "jsSearchButton") 
search_btn.click()

몇일 전 까지 크롤링할까요? 3


In [264]:
result_list

[[20251219, 'KRX 자동차', '2515.84', '12711'],
 [20251219, 'KRX 헬스케어', '4861.55', '29995'],
 [20251219, 'KRX 건설', '820.60', '35260'],
 [20251218, 'KRX 자동차', '2459.08', '9597'],
 [20251218, 'KRX 헬스케어', '4775.98', '21147'],
 [20251218, 'KRX 건설', '819.46', '61044'],
 [20251217, 'KRX 자동차', '2509.42', '20114'],
 [20251217, 'KRX 헬스케어', '4796.50', '25323'],
 [20251217, 'KRX 건설', '830.52', '55711']]

In [261]:
import pandas as pd
df = pd.DataFrame(result_list, columns=['날짜', '종목명', '종가(백만원)', '거래량(천주)'])
df

,날짜,종목명,종가(백만원),거래량(천주)
0,20251219,KRX 자동차,"2,515.84","12,711"
1,20251219,KRX 헬스케어,"4,861.55","29,995"
2,20251219,KRX 건설,820.60,"35,260"
3,20251218,KRX 자동차,"2,459.08","9,597"
4,20251218,KRX 헬스케어,"4,775.98","21,147"
5,20251218,KRX 건설,819.46,"61,044"
6,20251217,KRX 자동차,"2,509.42","20,114"
7,20251217,KRX 헬스케어,"4,796.50","25,323"
8,20251217,KRX 건설,830.52,"55,711"
9,20251216,KRX 자동차,"2,485.66","14,758"


In [262]:
df.to_csv('data/주식1주데이터.csv', index=False)